INITIALIZATION

In [ ]:
# Tải các thư viện
%pip install github pandas

In [1]:
# Nhập các thư viện
import pandas as pd
from github import Github
from datetime import datetime

GETTING REPO

In [2]:
# Khai báo token và repo cần phân tích
token_first_half = 'ghp_ckXuwpJIUBcDtI'
token_second_half = '6jF9cJ0KrUY4dyWH2FUqM4'
repo_owner = 'Anuken'
repo_name = 'Mindustry'

# Lấy repo
g = Github(token_first_half + token_second_half)
repo = g.get_repo(f"{repo_owner}/{repo_name}")

COLLECTING RAW DATA

In [3]:
# Thu thập dữ liệu về commits (trong năm 2023)
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31, 23, 59, 59)
commits = repo.get_commits(since= start_date, until = end_date)

# Thu thập dữ liệu về contributors
contributors = repo.get_contributors()

# Thu thập dữ liệu về issues
issues = repo.get_issues(state='all')

# Thu thập dữ liệu về pull requests
pulls = repo.get_pulls(state='all')

# Thu thập dữ liệu về programming languages
languages = repo.get_languages()

# Thu thập dữ liệu về files
contents = repo.get_contents("")


SAVING RAW DATA TO CSV DATAFRAMES

SAVING COMMITS

In [5]:
# Tạo bảng mới
commit_data = []

# Thêm dữ liệu vào bảng
for commit in commits:
    commit_files = [file.filename for file in commit.files]
    commit_data.append({
        'sha': commit.sha,
        'date': commit.commit.author.date,
        'author': commit.author.login if commit.author else 'unknown',
        'message': commit.commit.message,
        'files_changed': len(commit_files),
        'files_commited': commit_files
    })

# Lưu dữ liệu thô
pd.DataFrame(commit_data).to_csv('data_raw/commits.csv', index= False)

SAVING CONTRIBUTORS

In [ ]:
# Tạo bảng mới
contributors_data = []

# Thêm dữ liệu vào bảng
for contributor in contributors:
    contributors_data.append({
        'login': contributor.login,
        'commits': contributor.contributions,
        'location': contributor.location,
        'avatar_url': contributor.avatar_url
    })

# Lưu dữ liệu thô
pd.DataFrame(contributors_data).to_csv('data_raw/contributors.csv', index=False)

SAVING ISSUES

In [ ]:
# Tạo bảng mới
issues_data = []

# Thêm dữ liệu vào bảng
for issue in issues:
    if issue.pull_request:
        continue
    labels = [label.name for label in issue.labels]
    if not labels:
            labels = ['unlabeled']
    issues_data.append({
        'number': issue.number,
        'state': issue.state,
        'contributor': issue.user.login,
        'created_at': issue.created_at,
        'closed_at': issue.closed_at,
        'title': issue.title,
        'labels': labels
    })

# Lưu dữ liệu thô
pd.DataFrame(issues_data).to_csv('data_raw/issues.csv', index= False)

SAVING PULL REQUESTS

In [ ]:
# Tạo bảng mới
pulls_data = []

# Thêm dữ liệu vào bảng
for pull in pulls:
    pulls_data.append({
            'number': pull.number,
            'state': pull.state,
            'created_at': pull.created_at,
            'merged_at': pull.merged_at,
            'closed_at': pull.closed_at,
            'title': pull.title,
        })

# Lưu dữ liệu thô
pd.DataFrame(pulls_data).to_csv('data_raw/pull_requests.csv', index= False)

SAVING PROGRAMMING LANGUAGES

In [ ]:
# Tạo bảng mới
languages_data = []

# Thêm dữ liệu vào bảng
for language, lines in languages.items():
    languages_data.append({
        'language': language,
        'lines': lines
    })

# Lưu dữ liệu thô
pd.DataFrame(languages_data).to_csv('data_raw/languages.csv', index= False)

SAVING FILES

In [ ]:
# Tạo bảng mới
files_data = []

# Thêm dữ liệu vào bảng
while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(repo.get_contents(file_content.path))
    else:
        files_data.append({
            'path': file_content.path,
            'size': file_content.size,
        })

# Lưu dữ liệu thô
pd.DataFrame(files_data).to_csv('data_raw/files.csv', index= False)